In [12]:
import sys
import os
import pandas as pd
import essentia.standard as es
from pprint import pprint

# Set matplotlib to inline (if you plan on plotting later)
%matplotlib inline

# Determine the project root (assuming the notebook is in the notebooks folder)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
print("Project root:", project_root)

# Add the project root to sys.path so that the 'src' package is discoverable
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print("Current PYTHONPATH:", sys.path[:3])  # Verify that project_root is added

# Define paths to the raw audio folder and the model file
raw_dir = os.path.join(project_root, "data", "raw")
sample_audio = os.path.join(raw_dir, "example.mp3")  # Ensure this file exists
tempo_model_file = os.path.join(project_root, "src", "deeptemp-k16-3.pb")

print("Sample audio file:", sample_audio)
print("Tempo model file:", tempo_model_file)


Project root: /home/cepatinog/amplab
Current PYTHONPATH: ['/home/cepatinog/amplab', '/home/cepatinog/amplab/src', '/home/cepatinog/miniconda3/envs/amplab_essentia/lib/python310.zip']
Sample audio file: /home/cepatinog/amplab/data/raw/example.mp3
Tempo model file: /home/cepatinog/amplab/src/deeptemp-k16-3.pb


In [13]:
# Load the sample audio file at 44100 Hz (for key extraction)
audio_44100 = es.MonoLoader(filename=sample_audio, sampleRate=44100)()
print("Loaded audio length (samples):", len(audio_44100))

# For tempo extraction using TempoCNN, we need to resample to 11025 Hz.
audio_for_tempo = es.Resample(inputSampleRate=44100, outputSampleRate=11025)(audio_44100)
print("Resampled audio length (samples):", len(audio_for_tempo))


Loaded audio length (samples): 1323001
Resampled audio length (samples): 330750


In [14]:
# Import the extractor functions from our src folder.
# (Ensure your PYTHONPATH is set appropriately or run the notebook from the project root.)
from src.extract_tempo import extract_tempo_features
from src.key_extraction import extract_key_features

# Test tempo extraction using the 'tempocnn' method.
tempo_features = extract_tempo_features(audio_for_tempo, method='tempocnn', model_file=tempo_model_file)
print("Tempo Features:")
pprint(tempo_features)

# Test key extraction.
key_features = extract_key_features(audio_44100)  # using the 44100 Hz version for key extraction
print("\nKey Features:")
pprint(key_features)


Tempo Features:
{'global_bpm': 128.0,
 'local_bpms': [128.0, 128.0, 128.0, 128.0],
 'local_probs': [0.9546993374824524,
                 0.9014107584953308,
                 0.8928115367889404,
                 0.8102704882621765]}

Key Features:
{'edma_key': 'F#',
 'edma_scale': 'major',
 'edma_strength': 0.9310499429702759,
 'krumhansl_key': 'F#',
 'krumhansl_scale': 'major',
 'krumhansl_strength': 0.985231876373291,
 'temperley_key': 'F#',
 'temperley_scale': 'major',
 'temperley_strength': 0.9225367903709412}


2025-02-12 19:16:26.954592: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-12 19:16:26.954747: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.515GHz coreCount: 14 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 178.84GiB/s
2025-02-12 19:16:26.954859: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-12 19:16:26.954904: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-12 19:16:26.954916: I tensor